In [8]:
%load_ext autoreload
%autoreload 2

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="3" #! specify gpu here
import sys
import numpy as np
import cv2
import time
import torch
import clip
from PIL import Image
from rich import print
from tqdm import tqdm
from pathlib import Path
import csv
import json
import click

from vision_pipeline.llm_data_generator.labelme_importer import LabelMeImporter


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
qa_disassembly_step = {
    "Q": "What is the next disassembly step?",
    "A": "" #  "Turn the device over to show the back."
}
qa_relations = {
    "Q": "What are the object relations?",
    "A": "" # "PCB inside HCA, ..."
}
qa_positions = {
    "Q": "What are the object positions?",
    "A": "" # "HCA at (302, 300), PCB at (...), ..."
}
qa_json_disassembly_step = {
    "Q": "What is the JSON formatted disassembly step?",
    "A": {
        "reasoning": "",
        "action": "",
        "action_input": ""
    }
}

qas = [qa_disassembly_step, qa_relations, qa_positions, qa_json_disassembly_step]

In [10]:
kg_path = Path("~/datasets2/reconcycle/knowledge_graph").expanduser()
kg_nodes_path = kg_path / "data" / "nodes"
kg_edges_path = kg_path / "data" / "graph_edges.csv"

print("kg_nodes_path", kg_nodes_path)


kg_nodes_path /home/docker/datasets2/reconcycle/knowledge_graph/data/nodes

In [11]:
hcas = []
with open(kg_edges_path, 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        if len(row) >= 2 and row[1] == 'hca':
            hcas.append(row[0])

print(hcas)

['kalo', 'qundis']

In [15]:
# folder = kg_nodes_path / "kalo"
# folder = kg_nodes_path / "hekatron"
# folder = kg_nodes_path / "qundis"
folder = kg_nodes_path / "smoke_detector_2"

labelme_importer = LabelMeImporter()


# todo: get graph relations

# todo: get positions of objects 

from IPython.utils import io

# with io.capture_output() as captured:
img_paths, all_detections, all_graph_relations, modules, cameras, cropped_imgs = labelme_importer.process_labelme_dir(folder)


config:
{
    'node_name': 'vision',
    'reid': False,
    'basler': {
        'target_fps': 2,
        'max_allowed_acquisition_delay': 1.0,
        'rotate_img': 180,
        'topic': 'basler',
        'camera_node': '/basler',
        'image_topic': 'image_rect_color',
        'sleep_camera_on_exit': False,
        'publish_labelled_img': True,
        'publish_graph_img': True,
        'has_depth': False,
        'run_continuous': False,
        'wait_for_services': False,
        'detect_arucos': False,
        'parent_frame': 'vision_table_zero',
        'create_parent_frame': False,
        'marker_lifetime': 1,
        'work_surface_ignore_border_width': 100,
        'show_work_surface_detection': True,
        'use_worksurface_detection': True,
        'debug_work_surface_detection': False
    },
    'realsense': {
        'target_fps': 2,
        'max_allowed_acquisition_delay': 1.0,
        'rotate_img': 0,
        'topic': 'realsense',
        'camera_node': '/realsense',
        'image_topic': 'color/image_raw',
        'info_topic': 'color/camera_info',
        'depth_topic': 'aligned_depth_to_color/image_raw',
        'sleep_camera_on_exit': False,
        'publish_labelled_img': True,
        'publish_depth_img': True,
        'publish_cluster_img': True,
        'publish_graph_img': False,
        'has_depth': True,
        'run_continuous': False,
        'wait_for_services': False,
        'detect_arucos': False,
        'camera_height': 0.2,
        'parent_frame': 'panda_2/realsense',
        'create_parent_frame': False,
        'marker_lifetime': 1,
        'calibration_file': '~/vision_pipeline/realsense_calib/realsense_calib.yaml',
        'debug_clustering': False
    },
    'obj_detection': {
        'debug': True,
        'model': 'yolov8',
        'yolact_dataset_file': '~/vision_pipeline/data_limited/yolact/2023-07-18_firealarms_hcas/dataset.json',
        'yolact_score_threshold': 0.5,
        'yolov8_model_file': '~/vision_pipeline/data_limited/yolov8/output_2023-08-25_20000/best.pt',
        'yolov8_score_threshold': 0.5,
        'superglue_templates': '~/datasets2/reconcycle/2023-12-04_hcas_fire_alarms_sorted_cropped',
        'superglue_model_file': '~/superglue_training/output/train/2023-11-18_superglue_model/weights/best.pt',
        'superglue_match_threshold': 0.5,
        'rotation_median_filter': True,
        'classifier_model_file': 
'~/device_reid/results/2024-03-21__15-58_classify_WORKING/lightning_logs/version_0/checkpoints/epoch=212-step=212.c
kpt',
        'classifier_threshold': 0.5
    }
}

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


model.learning_rate 1e-05

model.batch_size 64

model.freeze_backbone False

...

elapsed_time_classify_and_rot 0.017544706352055073

graph: Device 1: battery in firealarm_front, pcb in firealarm_front, battery next to pcb.

list_wc_components
[{0, 1, 2}]

detections 3

Converting 4: 100%|██████████| 4/4 [00:00<00:00,  5.22it/s]


In [16]:
for img_path, detections, graph_relations, module, camera, cropped_img in zip(img_paths, all_detections, all_graph_relations, modules, cameras, cropped_imgs):

    stem = Path(img_path).stem
    qa_path = Path(img_path).parent / Path(str(stem) + "_qa.json")
    
    # save create cropped image
    cropped_img_name = str(stem) + "_crop.jpg"  
    cropped_img_path = Path(img_path).parent / Path(cropped_img_name)

    if not cropped_img_path.is_file() and cropped_img is not None:
      cv2.imwrite(str(cropped_img_path), cropped_img)

    print("qa_path", qa_path)

    A_relations = graph_relations.to_text()
    print("A_relations:", A_relations)

    # include the module name in the relations
    if module is not None:
        print("module", module.name)
        A_relations = f"Module: {module.name}. " + A_relations
    else:
        print("[red]Missing module name!")
    
    positions_list = []
    for detection in detections:
        if detection.label_precise is not None:
            label_id = detection.label_precise.split("_")[-1]
    
            positions_list.append(f"{detection.label.name} (id: {label_id}) at ({detection.center_px[0]}, {detection.center_px[1]})")
        else:
            positions_list.append(f"{detection.label.name} at ({detection.center_px[0]}, {detection.center_px[1]})")
        # print("detection name", detection.label.name)
        # print(f"detection center: {detection.center_px[0]}, {detection.center_px[1]}")
    
    A_positions = ", ".join(positions_list) + ". "

    print("A_positions:", A_positions)
    
    # create or edit the <number>_qa.json file
    if qa_path.is_file():
        print("[red]qa already exists!")
        
        with open(qa_path, 'r+') as f:
            data = json.load(f)
            data_changed = False
            for qa in data:
                if qa["Q"] == qa_relations["Q"] and qa["A"] != A_relations:
                    if click.confirm(f"overwrite QA relations\ncurrent: {qa['A']}\nnew: {A_relations}?\n", default=True):
                        qa["A"] = A_relations
                        data_changed = True
                elif qa["Q"] == qa_positions["Q"] and qa["A"] != A_positions:
                    if click.confirm(f"overwrite QA positions\ncurrent: {qa['A']}\nnew: {A_positions}?\n", default=True):
                        qa["A"] = A_positions
                        data_changed = True

            if data_changed:
                f.seek(0)        # <--- should reset file position to the beginning.
                json.dump(data, f, indent=4)
                f.truncate()     # remove remaining part

    else:
        qa_disassembly_step_copy = qa_disassembly_step.copy()
        qa_relations_copy = qa_relations.copy()
        qa_positions_copy = qa_positions.copy()
        qa_json_disassembly_step_copy = qa_json_disassembly_step.copy()

        qa_relations_copy["A"] = A_relations
        qa_positions_copy["A"] = A_positions

        QAs_copy = [qa_disassembly_step_copy, qa_relations_copy, qa_positions_copy, qa_json_disassembly_step_copy]

        with open(qa_path, 'w', encoding='utf-8') as f:
            json.dump(QAs_copy, f, ensure_ascii=False, indent=4)

qa_path /home/docker/datasets2/reconcycle/knowledge_graph/data/nodes/smoke_detector_2/1_qa.json

A_relations: Loose component: firealarm_back

module vision

A_positions: firealarm_back (id: 02) at (812, 688).

qa already exists!

...

qa_path /home/docker/datasets2/reconcycle/knowledge_graph/data/nodes/smoke_detector_2/4_qa.json

A_relations: Device 1: battery in firealarm_front, pcb in firealarm_front, battery next to pcb.

module cnc

A_positions: firealarm_front at (491, 491), battery at (466, 530), pcb at (507, 462).

qa already exists!

In [14]:

# todo: create QAs